In [ ]:
%%capture

import requests
import numpy as np
import xarray as xr
import dask
import dask.dataframe as dd
import pandas as pd
from dask.distributed import progress
from distributed import Client
from datetime import datetime, timedelta


def start_client():
    client = Client("dask-development-scheduler:8786")
    client.restart()
    return client


def download_data_ec(parameter, time_interval, format: str = "netcdf"):
    url = "http://smartmet.fmi.fi/download"
    payload = { "producer": "ecmwf_eurooppa_pinta",
                "format": format,
                "param": parameter,
                "starttime": time_interval["starttime"],
                "endtime": time_interval["endtime"]}
    r = requests.get(url, params=payload)
    data_ds = xr.open_dataset(r.content)
    return data_ds


def download_data_mos_archive(parameter, time_interval, format: str = "netcdf"):
    url = "http://smartmet.fmi.fi/download"
    payload = { "producer": "mos_eurooppa_pinta",
                "format": format,
                "param": parameter,
                "starttime": time_interval["starttime"],
                "endtime": time_interval["endtime"]}
    r = requests.get(url, params=payload)
    data_ds = xr.open_dataset(r.content)
    return data_ds
    

def download_data_meps(parameter, time_interval, format: str = "netcdf"):
    url = "http://smartmet.fmi.fi/download"
    payload = { "producer": "harmonie_skandinavia_pinta",
                "format": format,
                "param": parameter,
                "starttime": time_interval["starttime"],
                "endtime": time_interval["endtime"]}
    r = requests.get(url, params=payload)
    data_ds = xr.open_dataset(r.content)
    return data_ds


def download_data_gfs(parameter, time_interval, format: str = "netcdf"):
    url = "http://smartmet.fmi.fi/download"
    payload = { "producer": "gfs_scandinavia_surface",
                "format": format,
                "param": parameter,
                "starttime": time_interval["starttime"],
                "endtime": time_interval["endtime"]}
    r = requests.get(url, params=payload)
    data_ds = xr.open_dataset(r.content)
    return data_ds


def download_model_data(parameter, time_interval, format: str = "netcdf"):
    url = "http://smartmet.fmi.fi/download"
    payload = { "producer": "pal_skandinavia",
                "format": format,
                "param": parameter,
                "starttime": time_interval["starttime"],
                "endtime": time_interval["endtime"]}
    r = requests.get(url, params=payload)
    data_ds = xr.open_dataset(r.content)
    return data_ds
    

def get_model_data_delay(parameters, time_range, model):
    def generate_time_interval(time_range):
        now = datetime.now()
        time_intervals = []
        for i in range(time_range + 1):
            starttime = now + timedelta(days=i - 1)
            endtime = starttime + timedelta(days=1)
            time_interval =  { "starttime": datetime.strftime(starttime, "%Y-%m-%dT00:00:00"),
                               "endtime": datetime.strftime(endtime, "%Y-%m-%dT00:00:00") }
            time_intervals.append(time_interval)
        return time_intervals
    
    def merge_delayed(d):
        return xr.merge(d)

    if model == 'meps' and time_range > 3:
        time_range = 3
        
    time_intervals = generate_time_interval(time_range)
    if model == 'gfs':
        lots_of_data = [dask.delayed(download_data_gfs)(p, t) for p in parameters for t in time_intervals]
    elif model == 'meps':
        lots_of_data = [dask.delayed(download_data_meps)(p, t) for p in parameters for t in time_intervals]
    elif model == 'ec':
        lots_of_data = [dask.delayed(download_data_ec)(p, t) for p in parameters for t in time_intervals]
    else:
        lots_of_data = [dask.delayed(download_model_data)(p, t) for p in parameters for t in time_intervals]
    even_more_data = dask.delayed(merge_model_delayed)(d for d in lots_of_data)
    return even_more_data


def merge_model_delayed(client, even_more_data):
    merged = client.persist(even_more_data)
    progress(merged)
    print("Done!")
    return merged

In [ ]:
%%capture
%pip install matplotlib --upgrade --user
%pip install cartopy --upgrade --user

import requests
import numpy as np
import xarray as xr
import dask
import dask.dataframe as dd
import pandas as pd
from dask.distributed import progress
from distributed import Client
from datetime import datetime, timedelta
import ipywidgets as widgets
from matplotlib import pyplot as plt
import cartopy


def start_client():
    client = Client("dask-development-scheduler:8786")
    client.restart()
    return client


def download_data_ec(parameter, time_interval, format: str = "netcdf"):
    url = "http://smartmet.fmi.fi/download"
    payload = { "producer": "ecmwf_eurooppa_pinta",
                "format": format,
                "param": parameter,
                "starttime": time_interval["starttime"],
                "endtime": time_interval["endtime"]}
    r = requests.get(url, params=payload)
    data_ds = xr.open_dataset(r.content)
    return data_ds


def download_data_mos_archive(parameter, time_interval, format: str = "netcdf"):
    url = "http://smartmet.fmi.fi/download"
    payload = { "producer": "mos_eurooppa_pinta",
                "format": format,
                "param": parameter,
                "starttime": time_interval["starttime"],
                "endtime": time_interval["endtime"]}
    r = requests.get(url, params=payload)
    data_ds = xr.open_dataset(r.content)
    return data_ds
    

def download_data_meps(parameter, time_interval, format: str = "netcdf"):
    url = "http://smartmet.fmi.fi/download"
    payload = { "producer": "harmonie_skandinavia_pinta",
                "format": format,
                "param": parameter,
                "starttime": time_interval["starttime"],
                "endtime": time_interval["endtime"]}
    r = requests.get(url, params=payload)
    data_ds = xr.open_dataset(r.content)
    return data_ds


def download_data_gfs(parameter, time_interval, format: str = "netcdf"):
    url = "http://smartmet.fmi.fi/download"
    payload = { "producer": "gfs_scandinavia_surface",
                "format": format,
                "param": parameter,
                "starttime": time_interval["starttime"],
                "endtime": time_interval["endtime"]}
    r = requests.get(url, params=payload)
    data_ds = xr.open_dataset(r.content)
    return data_ds


def download_data(parameter, time_interval, format: str = "netcdf"):
    url = "http://smartmet.fmi.fi/download"
    payload = { "producer": "pal_skandinavia",
                "format": format,
                "param": parameter,
                "starttime": time_interval["starttime"],
                "endtime": time_interval["endtime"]}
    r = requests.get(url, params=payload)
    data_ds = xr.open_dataset(r.content)
    return data_ds
    

def get_model_data_delay(parameters, time_range, model):
    def generate_time_interval(time_range):
        now = datetime.now()
        time_intervals = []
        for i in range(time_range + 1):
            starttime = now + timedelta(days=i - 1)
            endtime = starttime + timedelta(days=1)
            time_interval =  { "starttime": datetime.strftime(starttime, "%Y-%m-%dT00:00:00"),
                               "endtime": datetime.strftime(endtime, "%Y-%m-%dT00:00:00") }
            time_intervals.append(time_interval)
        return time_intervals
    
    def merge_delayed(d):
        return xr.merge(d)

    if model == 'meps' and time_range > 3:
        time_range = 3
        
    time_intervals = generate_time_interval(time_range)
    if model == 'gfs':
        lots_of_data = [dask.delayed(download_data_gfs)(p, t) for p in parameters for t in time_intervals]
    elif model == 'meps':
        lots_of_data = [dask.delayed(download_data_meps)(p, t) for p in parameters for t in time_intervals]
    elif model == 'ec':
        lots_of_data = [dask.delayed(download_data_ec)(p, t) for p in parameters for t in time_intervals]
    else:
        lots_of_data = [dask.delayed(download_data)(p, t) for p in parameters for t in time_intervals]
    even_more_data = dask.delayed(merge_delayed)(d for d in lots_of_data)
    return even_more_data


def merge_delayed(client, even_more_data):
    merged = client.persist(even_more_data)
    progress(merged)
    print("Done!")
    return merged